<a href="https://colab.research.google.com/github/kuberiitb/artificial_intelligence/blob/main/rag/rag_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain langchain_openai langchain-community pypdf chromadb opentelemetry-api opentelemetry-sdk

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

import chromadb

from pprint import pprint
from IPython.display import Markdown, Image, display, HTML

In [ ]:
from google.colab import userdata
try:
  os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except:
  print("Set OPENAI_API_KEY")


In [ ]:
DOC_PATH = "/content/Building Effective AI Agents.pdf"

loader = PyPDFLoader(DOC_PATH)
pages = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
CHROMA_PATH = "/content/chroma_path"
os.makedirs(CHROMA_PATH,exist_ok=True)
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)

In [ ]:
query = 'What is AI Agent?'

query = 'What do frameworks do?'

docs_chroma = db_chroma.similarity_search_with_score(query, k=5)

context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

In [ ]:
print(context_text)

be implemented in a few lines of code. If you do use a framework, ensure you
understand the underlying code. Incorrect assumptions about what's under the
hood are a common source of customer error.
Skip to main contentSkip to footer

These frameworks make it easy to get started by simplifying standard low-level
tasks like calling LLMs, defining and parsing tools, and chaining calls together.
However, they often create extra layers of abstraction that can obscure the
underlying prompts   and responses, making them harder to debug. They can also
make it tempting to add complexity when a simpler setup would suffice.
We suggest that developers start by using LLM APIs directly: many patterns can

flexibility and model-driven decision-making are needed at scale. For many
applications, however, optimizing single LLM calls with retrieval and in-context
examples is usually enough.
When and how to use frameworks
There are many frameworks that make agentic systems easier to implement,
including:


In [ ]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
\n\n
Answer the question based on the above context: {question}.
Provide a detailed answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query)

In [ ]:
print(prompt)

Human: 
Answer the question based only on the following context:
be implemented in a few lines of code. If you do use a framework, ensure you
understand the underlying code. Incorrect assumptions about what's under the
hood are a common source of customer error.
Skip to main contentSkip to footer

These frameworks make it easy to get started by simplifying standard low-level
tasks like calling LLMs, defining and parsing tools, and chaining calls together.
However, they often create extra layers of abstraction that can obscure the
underlying prompts   and responses, making them harder to debug. They can also
make it tempting to add complexity when a simpler setup would suffice.
We suggest that developers start by using LLM APIs directly: many patterns can

flexibility and model-driven decision-making are needed at scale. For many
applications, however, optimizing single LLM calls with retrieval and in-context
examples is usually enough.
When and how to use frameworks
There are many fram

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",
                    temperature=0.2,
                    max_tokens=100)

response_text = llm.invoke(prompt)

In [ ]:
display(response_text.content)

'Frameworks simplify standard low-level tasks such as calling large language models (LLMs), defining and parsing tools, and chaining calls together. They make it easier to get started with implementing agentic systems. However, they often introduce extra layers of abstraction that can obscure the underlying prompts and responses, making debugging more challenging. Frameworks can also tempt developers to add unnecessary complexity when a simpler setup would suffice. While they help in getting started quickly, it is suggested to reduce abstraction layers and build with basic components'

In [ ]:
class DocAnswers:
  def __init__(self, model_name="gpt-4o-mini",
               DOC_PATH="/content/Building Effective AI Agents.pdf",
               ):
    self.llm = ChatOpenAI(model=model_name,
                    temperature=0.2,
                    max_tokens=10000)

    self.response_text = llm.invoke(prompt)
    self.DOC_PATH = DOC_PATH

    PROMPT_TEMPLATE = """
    Answer the question based only on the following context:
    {context}
    \n\n
    Answer the question based on the above context: {question}.
    If you can not find the answer in the context, reply "Sorry, I don't know that."
    Provide a short answer in 50-100 words.
    Don’t justify your answers.
    Don’t give information not mentioned in the CONTEXT INFORMATION.
    Do not say "according to the context" or "mentioned in the context" or similar.
    """

    self.prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

  def update_db(self):

    loader = PyPDFLoader(self.DOC_PATH)
    pages = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(pages)
    embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
    self.docs_chroma = db_chroma.similarity_search_with_score(query, k=5)

  def get_answer(self, query):
    context_text = "\n\n".join([doc.page_content for doc, _score in self.docs_chroma])
    self.prompt = prompt_template.format(context=context_text, question=query)

    self.response_text = llm.invoke(self.prompt)
    # print(self.response_text)
    return display(Markdown(self.response_text.content))


In [ ]:
answers = DocAnswers()

In [ ]:
answers.update_db()

In [ ]:
query = "What's the difference between a workflow and an agent?"

answers.get_answer(query)

A workflow is a system where LLMs and tools are orchestrated through predefined code paths. It provides predictability and consistency for well-defined tasks, allowing for the separation of concerns and the building of more specialized prompts. Workflows are particularly effective for complex tasks with distinct categories that are better handled separately.

An agent, on the other hand, is a system where LLMs dynamically direct their own processes and tool usage, maintaining control over how they accomplish tasks. Agents begin their work with a command

In [ ]:
query = "What's the difference between a workflow and an agent in one line?"

answers.get_answer(query)

A workflow is a system where LLMs and tools follow predefined code paths for task execution, while an agent is a system where LLMs dynamically direct their own processes and tool usage to accomplish tasks.

In [ ]:
answers.get_answer("What is India's capital city?")

The provided context does not contain any information regarding India's capital city. Therefore, I cannot provide an answer to that question.

In [ ]:
answers.get_answer("What is an AI Agent, tell in 2 short bullet points?")

- An AI Agent is a system where LLMs dynamically direct their own processes and tool usage, maintaining control over how they accomplish tasks.
- Agents begin their work with a command or interactive discussion with the human user, planning and operating independently while potentially returning for further information or judgment.

In [ ]:
questions = [
    "What's the difference between a workflow and an agent?",
    "What's the main idea behind the 'augmented LLM'?",
    "What is the 'prompt chaining' workflow and when should it be used?",
    "What is the purpose of the 'routing' workflow?",
    "What are the two main variations of the 'parallelization' workflow?",
    "How is the 'orchestrator-workers' workflow different from parallelization?",
    "What is the 'evaluator-optimizer' workflow and when is it effective?",
    "When should developers consider using a framework for building agentic systems?",
    "What are the main characteristics of an autonomous agent?",
    "What are the three core principles for implementing effective agents?"
]

In [ ]:
for question in questions:
  print(question)
  answers.get_answer(question)
  print("="*80)


What's the difference between a workflow and an agent?


A workflow is a system where LLMs and tools are orchestrated through predefined code paths. It provides predictability and consistency for well-defined tasks, allowing for the separation of concerns and the building of more specialized prompts. Workflows are suitable for complex tasks with distinct categories that are better handled separately.

An agent, on the other hand, is a system where LLMs dynamically direct their own processes and tool usage, maintaining control over how they accomplish tasks. Agents begin their work with a command from

What's the main idea behind the 'augmented LLM'?


The main idea behind the 'augmented LLM' is to enhance the capabilities of large language models (LLMs) by integrating them with tools and systems that allow for more dynamic and interactive processes. This involves two types of agentic systems: workflows and agents. 

Workflows are characterized by predefined code paths that orchestrate the LLMs and tools, providing predictability and consistency for well-defined tasks. They are particularly useful for complex tasks that require distinct categorization and specialized handling.

On the

What is the 'prompt chaining' workflow and when should it be used?


...

Developers should consider using a framework for building agentic systems when more complexity is warranted in their applications with LLMs. This is particularly relevant when the tasks require dynamic direction and control over processes and tool usage. Agentic systems are beneficial when handling complex inputs, engaging in reasoning and planning, and utilizing tools reliably. They are also suitable for scenarios where the system needs to recover from errors and maintain an interactive discussion with the human user to clarify tasks.

Additionally, developers should consider agentic systems when

What are the main characteristics of an autonomous agent?


An autonomous agent is characterized by the following main features:

1. **Dynamic Direction**: Agents have the ability to dynamically direct their own processes and tool usage, maintaining control over how they accomplish tasks.

2. **Initiation**: They begin their work with either a command from or an interactive discussion with the human user to clarify the task.

3. **Independent Operation**: Once the task is clear, agents can plan and operate independently without constant human intervention.

4. **Interaction and Adaptability

What are the three core principles for implementing effective agents?


The three core principles for implementing effective agents are:

1. **Dynamic Direction**: Agents should be capable of dynamically directing their own processes and tool usage, allowing them to maintain control over how they accomplish tasks.

2. **Interactive Engagement**: Agents begin their work with either a command from or interactive discussion with the human user, ensuring that the task is clear before they plan and operate independently.

3. **Tool Integration**: Agents should integrate tools effectively to enhance their capabilities, allowing them to access

Here we used recursive chunking RecursiveCharacterTextSplitter.

## Issues in the output:
- Output is not complete
- In one case, model did not retrieve anything at all, so we need to check the issue with Retriever.